<a href="https://colab.research.google.com/github/nurmukhammaddev/Audio_Module_NLP/blob/main/Spoken_Digit_Recognition_Youtube.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#ad the dataset
!wget https://github.com/Jakobovski/free-spoken-digit-dataset/archive/refs/heads/master.zip
#unzip the dataset folder
! unzip master.zip


--2023-11-09 09:44:36--  https://github.com/Jakobovski/free-spoken-digit-dataset/archive/refs/heads/master.zip
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/Jakobovski/free-spoken-digit-dataset/zip/refs/heads/master [following]
--2023-11-09 09:44:36--  https://codeload.github.com/Jakobovski/free-spoken-digit-dataset/zip/refs/heads/master
Resolving codeload.github.com (codeload.github.com)... 20.205.243.165
Connecting to codeload.github.com (codeload.github.com)|20.205.243.165|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘master.zip’

master.zip              [      <=>           ]  15.66M  15.1MB/s    in 1.0s    

2023-11-09 09:44:38 (15.1 MB/s) - ‘master.zip’ saved [16421388]

Archive:  master.zip
4c3b0983b1b7f46ec5e49ca527e8e8844a81107e
   creating: fre

In [2]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 76.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 13.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.7/302.7 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.8/395.8 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.4 

In [4]:
!unzip audio_uz.zip

unzip:  cannot find or open audio_uz.zip, audio_uz.zip.zip or audio_uz.zip.ZIP.


In [ ]:
import os
from os import listdir
from os.path import isfile, join
from matplotlib import pyplot as plt
import scipy.io.wavfile as wav
import numpy as np
from keras.preprocessing import image
from keras.utils import to_categorical


In [ ]:
def wav_to_spectrogram(audio_path, save_path, spectrogram_dimensions=(64, 64), noverlap=16, cmap='gray_r'):
  sample_rate, samples = wav.read(audio_path)
  fig = plt.figure()
  fig.set_size_inches((spectrogram_dimensions[0]/fig.get_dpi(), spectrogram_dimensions[1]/fig.get_dpi()))
  ax = plt.Axes(fig, [0., 0., 1., 1.])
  ax.set_axis_off()
  fig.add_axes(ax)
  ax.specgram(samples, cmap=cmap, Fs=2, noverlap=noverlap)
  ax.xaxis.set_major_locator(plt.NullLocator())
  ax.yaxis.set_major_locator(plt.NullLocator())
  fig.savefig(save_path, bbox_inches="tight", pad_inches=0)


In [ ]:
#
def dir_to_spectrogram(audio_dir, spectrogram_dir, spectrogram_dimensions=(64, 64), noverlap=16, cmap='gray_r'):
  file_names = [f for f in listdir(audio_dir) if isfile(join(audio_dir, f)) and '.wav' in f]
  for file_name in file_names:
    print(file_name)
    audio_path = audio_dir + file_name
    spectogram_path = spectrogram_dir + file_name.replace('.wav', '.png')
    wav_to_spectrogram(audio_path, spectogram_path, spectrogram_dimensions=spectrogram_dimensions, noverlap=noverlap, cmap=cmap)


In [ ]:
audio_folder = "audio_uz/"
spectrogram_folder = "spectrograms/"
dir_to_spectrogram(audio_folder, spectrogram_folder)


In [ ]:
imagesDir = "spectrograms/"
trainset = []
testset = []
for file in os.listdir(imagesDir):
  label = file.split('_')[0]
  sample_number = file.split('_')[2]
  img = image.load_img(imagesDir+file)
  if sample_number in ['0.png','1.png','2.png']:
    testset.append([image.img_to_array(img), label])
  else:
    trainset.append([image.img_to_array(img), label])


In [ ]:
#
# Get only images in the train list not the Labels
X_train = [item[0] for item in trainset]
# Get only Labels in the train list not the images
y_train = [item[1] for item in trainset]
# Get only images in the test list not the Labels
X_test = [item[0] for item in testset]
# Get only Labels in the test list not the images
y_test = [item[1] for item in testset]


In [ ]:
len(X_train)

854

In [ ]:
# Convert list to numpy array in order to define input shape
X_train = np.asanyarray(X_train)
y_train = np.asanyarray(y_train)
X_test = np.asanyarray(X_test)
y_test = np.asanyarray(y_test)


In [ ]:
# convert to one hot representation
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

#Normalize the images
X_train /= 255
X_test /= 255








In [ ]:
#N MODEL DESIGN

In [ ]:
from keras.models import Sequential
from keras import optimizers
from keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D, Dropout, BatchNormalization
from keras import models


In [ ]:
data_shape = (X_train.shape[1], X_train.shape[2], X_train.shape[3])
def basic_cnn():
  model = Sequential()
  model.add(Conv2D(32, kernel_size=(2, 2), activation='relu', input_shape=data_shape))
  model.add(BatchNormalization())
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Conv2D(16, kernel_size=(2, 2), activation='relu'))
  model.add(BatchNormalization())

  model.add(MaxPooling2D(pool_size=(3, 3)))
  model.add(Dropout(0.2))
  model.add(Flatten())
  model.add(Dense(64, activation='relu'))
  model.add(BatchNormalization())
  model.add(Dropout(0.35))
  model.add(Dense(32, activation='relu'))
  model.add(BatchNormalization())
  model.add(Dropout(0.4))
  model.add(Dense(10, activation='softmax'))
  model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model


In [ ]:
#
model0 = basic_cnn()
model0.summary()


Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_18 (Conv2D)          (None, 63, 63, 32)        416       
                                                                 
 batch_normalization_31 (Ba  (None, 63, 63, 32)        128       
 tchNormalization)                                               
                                                                 
 max_pooling2d_7 (MaxPoolin  (None, 31, 31, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_19 (Conv2D)          (None, 30, 30, 16)        2064      
                                                                 
 batch_normalization_32 (Ba  (None, 30, 30, 16)        64        
 tchNormalization)                                               
                                                      

In [ ]:
model0.fit(X_train, y_train, batch_size = 150, validation_split=0.2, epochs = 100, verbose = 1)


In [ ]:
model0.evaluate(X_test, y_test)

6/6 [==============================] - 0s 21ms/step - loss: 2.1609 - accuracy: 0.3294


[2.1609044075012207, 0.3294117748737335]

In [ ]:
!pip install pyyaml h5py  # Required to save models in HDF5 format
model0.save("spoken_digit_recognition_.h5")

In [ ]:
index = 23
print('ground Truth',np.argmax(y_test[index]))
print('Prediction' ,np.argmax(model0.predict(X_test[index].reshape(1,64,64,3))))

ground Truth 5
1/1 [==============================] - 0s 241ms/step
Prediction 5


In [ ]:
import gradio as gr

import numpy as np

from pydub import AudioSegment
import os
import scipy.io.wavfile as wav
import keras.utils as image
from keras.models import load_model
import matplotlib.pyplot as plt

import keras.utils as image

In [ ]:
def wav_to_spectrogram(audio_path, save_path, spectrogram_dimensions=(64, 64), noverlap=16, cmap='gray_r'):
  print(save_path, audio_path)
  sample_rate, samples = wav.read(audio_path)
  fig = plt.figure()
  fig.set_size_inches((spectrogram_dimensions[0]/fig.get_dpi(), spectrogram_dimensions[1]/fig.get_dpi()))
  ax = plt.Axes(fig, [0., 0., 1., 1.])
  ax.set_axis_off()
  fig.add_axes(ax)
  ax.specgram(samples, cmap=cmap, Fs=2, noverlap=noverlap)
  ax.xaxis.set_major_locator(plt.NullLocator())
  ax.yaxis.set_major_locator(plt.NullLocator())
  fig.savefig(save_path, bbox_inches="tight", pad_inches=0)

def predict_digit(audio_path,path_for_model):

    pid = os.getpid()

    rand = np.random.randint(1000000)

    spectrogram_path = f"predictions/{pid}_testing_{rand}.png"
    wav_to_spectrogram(audio_path, spectrogram_path)

    img = image.load_img(spectrogram_path)
    X = image.img_to_array(img)
    X = np.asanyarray(X)
    X /= 255
    # load model and predict

    #model_predict = load_model(path_for_model)

    return np.argmax(model0.predict(X.reshape(1,64,64,3)))


In [ ]:
def gradio_interface(audio_file):
    print(audio_file)

    return predict_digit(audio_path = audio_file,path_for_model='models/spoken_digit_recognition_12.h5')



In [ ]:
import gradio as gr
gr.Interface(fn=gradio_interface, inputs=gr.inputs.Audio(
            source="upload", type="filepath", optional=True, label="Record"
        ), outputs="text").launch(debug=True,share= True)

<ipython-input-32-4b94f5e3b9a1>:2: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  gr.Interface(fn=gradio_interface, inputs=gr.inputs.Audio(
<ipython-input-32-4b94f5e3b9a1>:2: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  gr.Interface(fn=gradio_interface, inputs=gr.inputs.Audio(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://75933f2ee616c9bc4d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


/tmp/gradio/f457ed304e775872b2bcecf523d0c918520963ed/0_AkmalAshirmatov_8-0-100.wav
predictions/662_testing_866832.png /tmp/gradio/f457ed304e775872b2bcecf523d0c918520963ed/0_AkmalAshirmatov_8-0-100.wav
1/1 [==============================] - 0s 30ms/step


/usr/local/lib/python3.10/dist-packages/gradio/processing_utils.py:188: UserWarning: Trying to convert audio automatically from int32 to 16-bit int format.
  warnings.warn(warning.format(data.dtype))


/tmp/gradio/72b2edc35293956478d8a950a34079d80ff4fb26/1_DiyoraSayfiddinova_12-0-100.wav
predictions/662_testing_643833.png /tmp/gradio/72b2edc35293956478d8a950a34079d80ff4fb26/1_DiyoraSayfiddinova_12-0-100.wav
1/1 [==============================] - 0s 30ms/step
/tmp/gradio/c2be4de4899d11ba934afe27a5766317dcbdb3b2/1_KamilaSaydullaeva_5-0-100.wav
predictions/662_testing_728271.png /tmp/gradio/c2be4de4899d11ba934afe27a5766317dcbdb3b2/1_KamilaSaydullaeva_5-0-100.wav
1/1 [==============================] - 0s 24ms/step
/tmp/gradio/7dfaf66bef08f5c60fb9f0b2758b548c83a94134/2_AmirSayfiddinov_2-0-100.wav
predictions/662_testing_737226.png /tmp/gradio/7dfaf66bef08f5c60fb9f0b2758b548c83a94134/2_AmirSayfiddinov_2-0-100.wav
1/1 [==============================] - 0s 39ms/step
/tmp/gradio/60e75429498ffe822f89d7ea8865781c89e703e9/2_AmirSayfiddinov_5-0-100.wav
predictions/662_testing_891141.png /tmp/gradio/60e75429498ffe822f89d7ea8865781c89e703e9/2_AmirSayfiddinov_5-0-100.wav
1/1 [=======================